In [53]:
import numpy as np
import csv
import sys
import os
import h5py
import pandas as pd
import simplejson as json
import sqlite3
import copy
from numpy import nan


In [43]:
sample_file = '../data/PAMAP2_Dataset/Protocol/subject101.dat'

datContent = [i.strip().split() for i in open(sample_file).readlines()]
datDF = pd.DataFrame(datContent)

In [68]:
all_data = {"data": {}, "target": {}}
prev_action = -1
starting = True
# action_seq = []
action_ID = 0
for l in open(sample_file).readlines():
    s = l.strip().split()
    if (prev_action != int(s[1])):
        if not(starting):
            all_data['data'][action_ID] = np.array(action_seq)
            all_data['target'][action_ID] = prev_action
            action_ID+=1
        action_seq = []
    else:
        starting = False
    data_seq = np.nan_to_num(np.array(s[3:]), nan=0)
    # data_seq[np.isnan(data_seq)] = 0
    action_seq.append(data_seq)
    prev_action = int(s[1])


In [72]:
all_data['data'][2].shape

(23480, 51)

In [44]:
datDF.head()

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,8.38,0,104,30,2.37223,8.60074,3.51048,2.43954,8.76165,3.35465,...,0.00830026,0.00925038,-0.0175803,-61.1888,-38.9599,-58.1438,1,0,0,0
1,8.39,0,NaN,30,2.18837,8.5656,3.66179,2.39494,8.55081,3.64207,...,-0.00657665,-0.00463778,0.00036825,-59.8479,-38.8919,-58.5253,1,0,0,0
2,8.4,0,NaN,30,2.37357,8.60107,3.54898,2.30514,8.53644,3.7328,...,0.00301426,0.000148236,0.022495,-60.7361,-39.4138,-58.3999,1,0,0,0
3,8.41,0,NaN,30,2.07473,8.52853,3.66021,2.33528,8.53622,3.73277,...,0.00317498,-0.0203009,0.0112754,-60.4091,-38.7635,-58.3956,1,0,0,0
4,8.42,0,NaN,30,2.22936,8.83122,3.7,2.23055,8.59741,3.76295,...,0.0126977,-0.0143027,-0.00282262,-61.5199,-39.3879,-58.2694,1,0,0,0


In [48]:
datDF.columns

RangeIndex(start=0, stop=54, step=1)

In [50]:
datDF.loc[datDF[1]=='0'].to_csv('../tmp/pamap2_act1.csv', index=False)

In [25]:
def readFile(file_path):
    all_data = {"data": {}, "target": {}, "collection": []}
    prev_action = -1
    starting = True
    # action_seq = []
    action_ID = 0
    for l in open(file_path).readlines():
        s = l.strip().split()
        if (prev_action != int(s[1])):
            if not(starting):
                all_data['data'][action_ID] = np.array(action_seq)
                all_data['target'][action_ID] = prev_action
                action_ID+=1
            action_seq = []
        else:
            starting = False
        data_seq = np.nan_to_num(np.array(s[3:]), nan=0)
        # data_seq[np.isnan(data_seq)] = 0
        action_seq.append(data_seq)
        prev_action = int(s[1])
        all_data['collection'].append(data_seq)
    return all_data

def readPamap2Files(data_path, filelist, cols, labelToId):
    data = []
    labels = []
    for i, filename in enumerate(filelist):
        print('Reading file %d of %d' % (i+1, len(filelist)))
        fpath = os.path.join(data_path, filename)
        file_data = readFile(fpath)
        data.extend(list(file_data['data'].values()))
        labels.extend(list(file_data['target'].values()))
    
    return {'inputs': np.asarray(data), 'targets': np.asarray(labels, dtype=int)+1}

def readPamap2(data_path):
    files = {
        'train': ['subject101.dat', 'subject102.dat','subject103.dat','subject104.dat', 'subject107.dat', 'subject108.dat', 'subject109.dat'],
        'test': ['subject106.dat']
    }
    label_map = [
        # (0, 'other'),
        (1, 'lying'),
        (2, 'sitting'),
        (3, 'standing'),
        (4, 'walking'),
        (5, 'running'),
        (6, 'cycling'),
        (7, 'Nordic walking'),
        (9, 'watching TV'),
        (10, 'computer work'),
        (11, 'car driving'),
        (12, 'ascending stairs'),
        (13, 'descending stairs'),
        (16, 'vacuum cleaning'),
        (17, 'ironing'),
        (18, 'folding laundry'),
        (19, 'house cleaning'),
        (20, 'playing soccer'),
        (24, 'rope jumping')
    ]
    labelToId = {str(x[0]): i for i, x in enumerate(label_map)}
    # print "label2id=",labelToId
    idToLabel = [x[1] for x in label_map]
    # print "id2label=",idToLabel
    cols = [
            1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
            35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53
            ]
    # print "cols",cols
    data = {dataset: readPamap2Files(data_path, files[dataset], cols, labelToId)
            for dataset in ('train', 'test')}
    return data, idToLabel

In [26]:
pam_data = readPamap2('../data/PAMAP2_Dataset/Protocol/')

Reading file 1 of 7
Reading file 2 of 7
Reading file 3 of 7
Reading file 4 of 7
Reading file 5 of 7
Reading file 6 of 7
Reading file 7 of 7


C:\Users\deela\AppData\Local\Temp\ipykernel_29436\2408505951.py:33: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return {'inputs': np.asarray(data), 'targets': np.asarray(labels, dtype=int)+1}


Reading file 1 of 1


In [27]:
pam_data[0]['train']['inputs'].shape

(142,)

In [28]:
pam_data[1]

['lying',
 'sitting',
 'standing',
 'walking',
 'running',
 'cycling',
 'Nordic walking',
 'watching TV',
 'computer work',
 'car driving',
 'ascending stairs',
 'descending stairs',
 'vacuum cleaning',
 'ironing',
 'folding laundry',
 'house cleaning',
 'playing soccer',
 'rope jumping']

In [29]:
train_data = pam_data[0]['train']
test_data = pam_data[0]['test']
# num_i, num_c = train_data['inputs'].shape 
classes = pam_data[1]


In [30]:
train_vals, train_labels = train_data['inputs'], train_data['targets']
train_labels

array([ 1,  2,  3,  4,  1, 18,  1, 17,  1, 13,  1, 14,  1, 13, 14,  1,  5,
        1,  8,  1,  7,  1,  6,  1, 25,  1,  2,  1,  3,  1,  4,  1, 18,  1,
       17,  1, 13,  1, 14,  1, 13, 14,  1,  5,  1,  8,  1,  7,  1,  6,  1,
       25,  1,  2,  1,  3,  4, 18,  1, 17,  1, 14,  1, 13, 14,  1, 13, 14,
        1,  5,  1,  2,  1,  3,  4, 18,  1, 17,  1, 13,  1, 14,  1, 13, 14,
        1,  5,  1,  8,  1,  7,  1,  1,  2,  1,  3,  1,  4, 18,  1, 17,  1,
       13,  1, 14,  1, 13, 14,  1,  5,  1,  8,  1,  7,  1,  1,  2,  1,  3,
        4, 18,  1, 17,  1, 13,  1, 14,  1, 13, 14,  1,  5,  1,  7,  1,  8,
        1,  6,  1, 25,  1, 25])

In [31]:
filtered_classes = [1, 4, 6, 18]
flt_mask = np.in1d(train_labels, filtered_classes)
flt_mask

array([ True, False, False,  True,  True,  True,  True, False,  True,
       False,  True, False,  True, False, False,  True, False,  True,
       False,  True, False,  True,  True,  True, False,  True, False,
        True, False,  True,  True,  True,  True,  True, False,  True,
       False,  True, False,  True, False, False,  True, False,  True,
       False,  True, False,  True,  True,  True, False,  True, False,
        True, False,  True,  True,  True, False,  True, False,  True,
       False, False,  True, False, False,  True, False,  True, False,
        True, False,  True,  True,  True, False,  True, False,  True,
       False,  True, False, False,  True, False,  True, False,  True,
       False,  True,  True, False,  True, False,  True,  True,  True,
        True, False,  True, False,  True, False,  True, False, False,
        True, False,  True, False,  True, False,  True,  True, False,
        True, False,  True,  True,  True, False,  True, False,  True,
       False,  True,

In [33]:
np.invert(flt_mask)

array([False,  True,  True, False, False, False, False,  True, False,
        True, False,  True, False,  True,  True, False,  True, False,
        True, False,  True, False, False, False,  True, False,  True,
       False,  True, False, False, False, False, False,  True, False,
        True, False,  True, False,  True,  True, False,  True, False,
        True, False,  True, False, False, False,  True, False,  True,
       False,  True, False, False, False,  True, False,  True, False,
        True,  True, False,  True,  True, False,  True, False,  True,
       False,  True, False, False, False,  True, False,  True, False,
        True, False,  True,  True, False,  True, False,  True, False,
        True, False, False,  True, False,  True, False, False, False,
       False,  True, False,  True, False,  True, False,  True,  True,
       False,  True, False,  True, False,  True, False, False,  True,
       False,  True, False, False, False,  True, False,  True, False,
        True, False,

In [88]:
flt_label = train_labels[flt_mask]
flt_data = train_vals[flt_mask]
print(flt_data.shape, flt_label.shape)

(28,) (28,)


In [25]:
# build sample ZSL training dataloader
import numpy as np
from sklearn.model_selection import train_test_split
import random
from scipy.signal import resample

class PAMAP2Reader(object):
    def __init__(self, root_path):
        self.root_path = root_path
        self.readPamap2()

    def readFile(self, file_path):
        all_data = {"data": {}, "target": {}}
        prev_action = -1
        starting = True
        # action_seq = []
        action_ID = 0
        for l in open(file_path).readlines():
            s = l.strip().split()
            if s[1] != "0": # skip intermediate records
                if (prev_action != int(s[1])):
                    if not(starting):
                        all_data['data'][action_ID] = np.array(action_seq)
                        all_data['target'][action_ID] = prev_action
                        action_ID+=1
                    action_seq = []
                else:
                    starting = False
                data_seq = np.nan_to_num(np.array(s[3:]), nan=0)
                # data_seq[np.isnan(data_seq)] = 0
                action_seq.append(data_seq)
                prev_action = int(s[1])
        return all_data

    def readPamap2Files(self, filelist, cols, labelToId):
        data = []
        labels = []
        for i, filename in enumerate(filelist):
            print('Reading file %d of %d' % (i+1, len(filelist)))
            fpath = os.path.join(self.root_path, filename)
            file_data = self.readFile(fpath)
            data.extend(list(file_data['data'].values()))
            labels.extend(list(file_data['target'].values()))
        
        return np.asarray(data), np.asarray(labels, dtype=int)

    def readPamap2(self):
        files = ['subject101.dat', 'subject102.dat'] #['subject101.dat', 'subject102.dat','subject103.dat','subject104.dat', 'subject105.dat', 'subject106.dat', 'subject107.dat', 'subject108.dat', 'subject109.dat']
            
        label_map = [
            # (0, 'other'),
            (1, 'lying'),
            (2, 'sitting'),
            (3, 'standing'),
            (4, 'walking'),
            (5, 'running'),
            (6, 'cycling'),
            (7, 'Nordic walking'),
            (9, 'watching TV'),
            (10, 'computer work'),
            (11, 'car driving'),
            (12, 'ascending stairs'),
            (13, 'descending stairs'),
            (16, 'vacuum cleaning'),
            (17, 'ironing'),
            (18, 'folding laundry'),
            (19, 'house cleaning'),
            (20, 'playing soccer'),
            (24, 'rope jumping')
        ]
        labelToId = {x[0]: i for i, x in enumerate(label_map)}
        # print "label2id=",labelToId
        idToLabel = [x[1] for x in label_map]
        # print "id2label=",idToLabel
        cols = [
                1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
                35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53
                ]
        # print "cols",cols
        self.data, self.targets = self.readPamap2Files(files, cols, labelToId)
        f = lambda x: labelToId[x]
        print("labelToId > ", labelToId)
        self.targets = np.array([labelToId[i] for i in list(self.targets)]) #np.vectorize(labelToId.get)(self.targets)
        self.label_map = label_map
        self.idToLabel = idToLabel
        # return data, idToLabel

    def resampling(self, data, targets):
        assert len(data) == len(targets), "# action data & # action labels are not matching"
        all_data, all_ids, all_labels = [], [], []
        for i, d in enumerate(data):
            l, _ = d.shape 
            label = targets[i]
            if l < 1200 : # minimum length requirement
                break 
            # generate sampling points
            n_point, _ = divmod(l, 1200)
            sampling_points = np.arange(start=0, stop=l, step=1200)[:-1]
            sampling_points = sampling_points-200
            sampling_points[0] = 0
            # window sampling 
            for s in sampling_points:
                sub_sample = resample(d[s:s+1200, :], num=120)
                all_data.append(sub_sample)
                all_ids.append(i+1)
                all_labels.append(label)
            
        return all_data, all_ids, all_labels

    def generate(self, unseen_classes, resampling=True, seen_ratio=0.2, unseen_ratio=0.8):
        # assert all([i in list(self.label_map.keys()) for i in unseen_classes]), "Unknown Class label!"
        seen_classes = [i for i in self.idToLabel if i not in unseen_classes]
        unseen_mask = np.in1d(self.targets, unseen_classes)

        # build seen dataset 
        seen_data = self.data[np.invert(unseen_mask)]
        seen_targets = self.targets[np.invert(unseen_mask)]

        # build unseen dataset
        unseen_data = self.data[unseen_mask]
        unseen_targets = self.targets[unseen_mask]

        # resampling seen and unseen datasets 
        seen_data, seen_ids, seen_targets = self.resampling(seen_data, seen_targets)
        unseen_data, unseen_ids, unseen_targets = self.resampling(unseen_data, unseen_targets)

        seen_data, seen_targets = np.array(seen_data), np.array(seen_targets)
        unseen_data, unseen_targets = np.array(unseen_data), np.array(unseen_targets)
        # train-val split
        seen_index = list(range(len(seen_targets)))
        random.shuffle(seen_index)
        split_point = int((1-seen_ratio)*len(seen_index))
        fst_index, sec_index = seen_index[:split_point], seen_index[split_point:]
        print(type(fst_index), type(sec_index), type(seen_data), type(seen_targets))
        X_seen_train, X_seen_val, y_seen_train, y_seen_val = seen_data[fst_index,:], seen_data[sec_index,:], seen_targets[fst_index], seen_targets[sec_index]
        
        # val-test split
        unseen_index = list(range(len(unseen_targets)))
        random.shuffle(unseen_index)
        split_point = int((1-unseen_ratio)*len(unseen_index))
        fst_index, sec_index = unseen_index[:split_point], unseen_index[split_point:]

        X_unseen_val, X_unseen_test, y_unseen_val, y_unseen_test = unseen_data[fst_index,:], unseen_data[sec_index,:], unseen_targets[fst_index], unseen_targets[sec_index]

        data = {'train': {
                        'X': X_seen_train,
                        'y': y_seen_train
                        },
                'eval-seen':{
                        'X': X_seen_val,
                        'y': y_seen_val
                        },
                'eval-unseen':{
                        'X': X_unseen_val,
                        'y': y_unseen_val
                        },
                'test': {
                        'X': X_unseen_test,
                        'y': y_unseen_test
                        }
                }

        return data
        

In [26]:
dataReader = PAMAP2Reader('../data/PAMAP2_Dataset/Protocol/')

Reading file 1 of 2
Reading file 2 of 2


C:\Users\deela\AppData\Local\Temp\ipykernel_37144\708394372.py:45: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(data), np.asarray(labels, dtype=int)


labelToId >  {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 9: 7, 10: 8, 11: 9, 12: 10, 13: 11, 16: 12, 17: 13, 18: 14, 19: 15, 20: 16, 24: 17}


In [27]:
dataReader.targets

array([ 0,  1,  2, 13, 12, 10, 11, 10, 11,  3,  6,  5,  4,  0,  1,  2, 13,
       12, 10, 11, 10, 11,  3,  6,  5,  4])

In [64]:
data_dict = dataReader.generate(unseen_classes=[2, 4, 14, 18], resampling=True, seen_ratio=0.2, unseen_ratio=0.8)

<class 'list'> <class 'list'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [65]:
data_dict['train']['X'].shape

(388, 120, 51)

In [66]:
data_dict['train']['y'].shape

(388,)

In [67]:
dataReader.idToLabel

['lying',
 'sitting',
 'standing',
 'walking',
 'running',
 'cycling',
 'Nordic walking',
 'watching TV',
 'computer work',
 'car driving',
 'ascending stairs',
 'descending stairs',
 'vacuum cleaning',
 'ironing',
 'folding laundry',
 'house cleaning',
 'playing soccer',
 'rope jumping']

---

In [9]:
print(type(dataReader.data), len(dataReader.data))

<class 'numpy.ndarray'> 190


In [ ]:
[i.shape for i in dataReader.data]

In [11]:
divmod(12, 7)

(1, 5)

In [12]:
np.arange(start=0, stop=6391, step=1200)

array([   0, 1200, 2400, 3600, 4800, 6000])

In [15]:
# simplest sampling strategy 
# frequency : 10Hz (human-eye sensitivity) 
# sequence length : 120 points
# expected minimum length : 1200
# window sampling method : shifting window method with 200 point overlapping
from scipy.signal import resample

def signal_sampling(data, labels):
    assert len(data) == len(labels), "# action data & # action labels are not matching"
    all_data, all_ids, all_labels = [], [], []
    for i, d in enumerate(data):
        l, _ = d.shape 
        label = labels[i]
        if l < 1200 : # minimum length requirement
            break 
        # generate sampling points
        n_point, _ = divmod(l, 1200)
        sampling_points = np.arange(start=0, stop=l, step=1200)[:-1]
        sampling_points = sampling_points-200
        sampling_points[0] = 0
        # window sampling 
        for s in sampling_points:
            sub_sample = resample(d[s:s+1200, :], num=120)
            all_data.append(sub_sample)
            all_ids.append(i+1)
            all_labels.append(label)
        
    return all_data, all_ids, all_labels

In [16]:
all_data, all_ids, all_labels = signal_sampling(dataReader.data, dataReader.targets)

In [18]:
len(all_data), len(all_ids), all_data[0].shape

(768, 768, (120, 51))

In [58]:
all_data[[12,4,0]]

TypeError: list indices must be integers or slices, not list

In [ ]:
import torch
from torch.utils.data import Dataset



In [68]:
fold_classes = [['watching TV', 'house cleaning', 'standing', 'ascending stairs'], ['walking', 'rope jumping', 'sitting', 'descending stairs'], ['playing soccer', 'lying', 'vacuum cleaning', 'computer work'], ['cycling', 'running', 'Nordic walking'], ['ironing', 'car driving', 'folding laundry']]

---
Load Attribute Info

In [74]:
import json 
import numpy as np

def load_attribute(fpath):
    with open(fpath, "r") as pf:
        json_data = json.load(pf)

    activity_dict = json_data["Activity"]
    attribute_dict = json_data["Attribute"]
    attr_met = np.array(list(json_data["Mark"].values()))
    return activity_dict, attribute_dict, attr_met

In [75]:
a,b,c = load_attribute('./PAMAP2_attributes.json')

In [77]:
c.shape

(18, 42)

In [79]:
import numpy as np

feat_size = 64
n_actions = 18 
feat_mat = np.random.randn(n_actions, feat_size)

In [80]:
feat_mat.shape

(18, 64)

---

In [1]:
import os
import json
import random 
import math
import numpy as np
from tqdm import tqdm
from copy import deepcopy

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, Tensor
from torch.nn import functional as F 
from torch.optim import Adam

from scipy.signal import resample
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

In [23]:
# build PAMAP2 dataset data reader
class PAMAP2Reader(object):
    def __init__(self, root_path):
        self.root_path = root_path
        self.readPamap2()

    def readFile(self, file_path):
        all_data = {"data": {}, "target": {}, 'collection': []}
        prev_action = -1
        starting = True
        # action_seq = []
        action_ID = 0
        for l in open(file_path).readlines():
            s = l.strip().split()
            if s[1] != "0":
                if (prev_action != int(s[1])):
                    if not(starting):
                        all_data['data'][action_ID] = np.array(action_seq)
                        all_data['target'][action_ID] = prev_action
                        action_ID+=1
                    action_seq = []
                else:
                    starting = False
                data_seq = np.nan_to_num(np.array(s[3:]), nan=0).astype(np.float16)
                # data_seq[np.isnan(data_seq)] = 0
                action_seq.append(data_seq)
                prev_action = int(s[1])
                all_data['collection'].append(data_seq)
        return all_data

    def readPamap2Files(self, filelist, cols, labelToId):
        data = []
        labels = []
        collection = []
        for i, filename in enumerate(filelist):
            print('Reading file %d of %d' % (i+1, len(filelist)))
            fpath = os.path.join(self.root_path, filename)
            file_data = self.readFile(fpath)
            data.extend(list(file_data['data'].values()))
            labels.extend(list(file_data['target'].values()))
            collection.extend(file_data['collection'])
        return np.asarray(data), np.asarray(labels, dtype=int), np.array(collection)

    def readPamap2(self):
        files = ['subject101.dat', 'subject102.dat','subject103.dat','subject104.dat', 'subject105.dat', 'subject106.dat', 'subject107.dat', 'subject108.dat', 'subject109.dat']
            
        label_map = [
            #(0, 'other'),
            (1, 'lying'),
            (2, 'sitting'),
            (3, 'standing'),
            (4, 'walking'),
            (5, 'running'),
            (6, 'cycling'),
            (7, 'Nordic walking'),
            (9, 'watching TV'),
            (10, 'computer work'),
            (11, 'car driving'),
            (12, 'ascending stairs'),
            (13, 'descending stairs'),
            (16, 'vacuum cleaning'),
            (17, 'ironing'),
            (18, 'folding laundry'),
            (19, 'house cleaning'),
            (20, 'playing soccer'),
            (24, 'rope jumping')
        ]
        labelToId = {x[0]: i for i, x in enumerate(label_map)}
        # print "label2id=",labelToId
        idToLabel = [x[1] for x in label_map]
        # print "id2label=",idToLabel
        cols = [
                1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
                35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53
                ]
        # print "cols",cols
        self.data, self.targets, self.all_data = self.readPamap2Files(files, cols, labelToId)
        # print(self.data)
        # nan_perc = np.isnan(self.data).astype(int).mean()
        # print("null value percentage ", nan_perc)
        # f = lambda x: labelToId[x]
        self.targets = np.array([labelToId[i] for i in list(self.targets)])
        self.label_map = label_map
        self.idToLabel = idToLabel
        # return data, idToLabel

    def resampling(self, data, targets):
        assert len(data) == len(targets), "# action data & # action labels are not matching"
        all_data, all_ids, all_labels = [], [], []
        for i, d in enumerate(data):
            l, _ = d.shape 
            label = targets[i]
            if l < 1200 : # minimum length requirement
                break 
            # generate sampling points
            n_point, _ = divmod(l, 1200)
            sampling_points = np.arange(start=0, stop=l, step=1200)[:-1]
            sampling_points = sampling_points-200
            sampling_points[0] = 0
            # print(l, sampling_points)
            # window sampling 
            for s in sampling_points:
                sub_sample = np.nan_to_num(resample(d[s:s+1200, :], num=120, axis=0), nan=0)
                # print(d[s:s+1200, :])
                nan_perc = np.isnan(sub_sample).astype(int).mean()
                print("null value percentage ", nan_perc, "id > ", i+1)
                all_data.append(sub_sample)
                all_ids.append(i+1)
                all_labels.append(label)
            
        return all_data, all_ids, all_labels

    def generate(self, unseen_classes, resampling=True, seen_ratio=0.2, unseen_ratio=0.8):
        # assert all([i in list(self.label_map.keys()) for i in unseen_classes]), "Unknown Class label!"
        seen_classes = [i for i in range(len(self.idToLabel)) if i not in unseen_classes]
        unseen_mask = np.in1d(self.targets, unseen_classes)

        # normalize data [without considering ]
        # scaler = MinMaxScaler()
        # scaler.fit(self.all_data)
        # self.data = np.array([scaler.transform(d) for d in self.data])

        # build seen dataset 
        seen_data = self.data[np.invert(unseen_mask)]
        seen_targets = self.targets[np.invert(unseen_mask)]

        # build unseen dataset
        unseen_data = self.data[unseen_mask]
        unseen_targets = self.targets[unseen_mask]

        # resampling seen and unseen datasets 
        seen_data, seen_ids, seen_targets = self.resampling(seen_data, seen_targets)
        unseen_data, unseen_ids, unseen_targets = self.resampling(unseen_data, unseen_targets)

        seen_data, seen_targets = np.array(seen_data), np.array(seen_targets)
        unseen_data, unseen_targets = np.array(unseen_data), np.array(unseen_targets)
        # train-val split
        seen_index = list(range(len(seen_targets)))
        random.shuffle(seen_index)
        split_point = int((1-seen_ratio)*len(seen_index))
        fst_index, sec_index = seen_index[:split_point], seen_index[split_point:]
        print(type(fst_index), type(sec_index), type(seen_data), type(seen_targets))
        X_seen_train, X_seen_val, y_seen_train, y_seen_val = seen_data[fst_index,:], seen_data[sec_index,:], seen_targets[fst_index], seen_targets[sec_index]
        
        # val-test split
        unseen_index = list(range(len(unseen_targets)))
        random.shuffle(unseen_index)
        split_point = int((1-unseen_ratio)*len(unseen_index))
        fst_index, sec_index = unseen_index[:split_point], unseen_index[split_point:]

        X_unseen_val, X_unseen_test, y_unseen_val, y_unseen_test = unseen_data[fst_index,:], unseen_data[sec_index,:], unseen_targets[fst_index], unseen_targets[sec_index]

        data = {'train': {
                        'X': X_seen_train,
                        'y': y_seen_train
                        },
                'eval-seen':{
                        'X': X_seen_val,
                        'y': y_seen_val
                        },
                'eval-unseen':{
                        'X': X_unseen_val,
                        'y': y_unseen_val
                        },
                'test': {
                        'X': X_unseen_test,
                        'y': y_unseen_test
                        },
                'seen_classes': seen_classes,
                'unseen_classes': unseen_classes
                }

        return data
        

In [3]:
# import action attributes
def load_attribute(fpath):
    with open(fpath, "r") as pf:
        json_data = json.load(pf)

    activity_dict = json_data["Activity"]
    attribute_dict = json_data["Attribute"]
    attr_met = np.array(list(json_data["Mark"].values()))
    return activity_dict, attribute_dict, attr_met

In [4]:
# import action semantic features 
# generate random feature matrix 
feat_size = 64
n_actions = 18 
feat_mat = np.random.randn(n_actions, feat_size)

In [5]:
# build mock dataloader
class PAMAP2Dataset(Dataset):
    def __init__(self, data, actions, attributes, action_feats, action_classes):
        super(PAMAP2Dataset, self).__init__()
        self.data = torch.from_numpy(data)
        self.actions = actions
        self.attributes = torch.from_numpy(attributes)
        self.action_feats = torch.from_numpy(action_feats)
        self.target_feat = torch.from_numpy(action_feats[action_classes, :])
        # build action to id mapping dict
        self.n_action = len(self.actions)
        self.action2Id = dict(zip(action_classes, range(self.n_action)))

    def __getitem__(self, ind):
        x = self.data[ind, ...]
        x_mask = np.array([0]) #self.padding_mask[ind, ...]
        target = self.actions[ind]
        y = torch.from_numpy(np.array([self.action2Id[target]]))
        y_feat = self.action_feats[target, ...]
        attr = self.attributes[target, ...]
        return x, y, y_feat, attr, x_mask

    def __len__(self):
        return self.data.shape[0]


In [24]:
# import PAMAP2 dataset
dataReader = PAMAP2Reader('../data/PAMAP2_Dataset/Protocol/')
actionList = dataReader.idToLabel
# import PAMAP2 attributes
activity_dict, attribute_dict, attr_mat = load_attribute('./PAMAP2_attributes.json')
_, attr_size = attr_mat.shape
# import action features
feat_size = 64
n_actions = 18 
feat_mat = np.random.randn(n_actions, feat_size) 

# run 5-fold running
fold_classes = [['watching TV', 'house cleaning', 'standing', 'ascending stairs'], ['walking', 'rope jumping', 'sitting', 'descending stairs'], ['playing soccer', 'lying', 'vacuum cleaning', 'computer work'], ['cycling', 'running', 'Nordic walking'], ['ironing', 'car driving', 'folding laundry']]
fold_cls_ids = [[actionList.index(i) for i in j] for j in fold_classes]

# separate seen/unseen and train/eval 
data_dict = dataReader.generate(unseen_classes=[1, 4, 14, 2], resampling=True, seen_ratio=0.2, unseen_ratio=0.8)
all_classes = dataReader.idToLabel
seen_classes = data_dict['seen_classes']
unseen_classes = data_dict['unseen_classes']
train_n, seq_len, in_ft = data_dict['train']['X'].shape
# build train_dt 
train_dt = PAMAP2Dataset(data=data_dict['train']['X'], actions=data_dict['train']['y'], attributes=attr_mat, action_feats=feat_mat, action_classes=seen_classes)
train_dl = DataLoader(train_dt, batch_size=32, shuffle=True, pin_memory=True)


Reading file 1 of 9
Reading file 2 of 9
Reading file 3 of 9
Reading file 4 of 9
Reading file 5 of 9
Reading file 6 of 9
Reading file 7 of 9
Reading file 8 of 9
Reading file 9 of 9


C:\Users\deela\AppData\Local\Temp\ipykernel_9008\3001450184.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(data), np.asarray(labels, dtype=int), np.array(collection)


null value percentage  0.0 id >  1
null value percentage  0.0 id >  1
null value percentage  0.0 id >  1
null value percentage  0.0 id >  1
null value percentage  0.0 id >  1
null value percentage  0.0 id >  1
null value percentage  0.0 id >  1
null value percentage  0.0 id >  1
null value percentage  0.0 id >  1
null value percentage  0.0 id >  1
null value percentage  0.0 id >  1
null value percentage  0.0 id >  1
null value percentage  0.0 id >  1
null value percentage  0.0 id >  1
null value percentage  0.0 id >  1
null value percentage  0.0 id >  1
null value percentage  0.0 id >  1
null value percentage  0.0 id >  1
null value percentage  0.0 id >  1
null value percentage  0.0 id >  1
null value percentage  0.0 id >  1
null value percentage  0.0 id >  1
null value percentage  0.0 id >  2
null value percentage  0.0 id >  2
null value percentage  0.0 id >  2
null value percentage  0.0 id >  2
null value percentage  0.0 id >  2
null value percentage  0.0 id >  2
null value percentag

In [21]:
a = np.array([1, .43, 32.43, np.nan, 32])
b = np.nan_to_num(a, nan=-1)
b

array([ 1.  ,  0.43, 32.43, -1.  , 32.  ])

In [20]:
train_data = data_dict['train']['X']
np.isnan(train_data).astype(int).mean()

0.44332961724266073

In [11]:
for x,y,y_feat,attr,mask in train_dt:
    print(torch.isnan(x))


tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])
tensor([[False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        ...,
        [False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True]])
tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [